https://velog.io/@jaehyeong/CNN-%EB%AA%A8%EB%8D%B8%EC%9D%84-%ED%86%B5%ED%95%9C-%EC%9E%90%EB%8F%99%EC%B0%A8-%EC%82%AC%EA%B3%A0-%EC%9D%B4%EB%AF%B8%EC%A7%80-%EB%B6%84%EB%A5%98

In [1]:
import numpy as np
import pandas as pd
import os, glob
from PIL import Image
from sklearn.model_selection import train_test_split

import tensorflow as tf
from keras.utils import set_random_seed

In [2]:
# 랜덤값 고정
np.random.seed(42)
tf.random.set_seed(42)
set_random_seed(42)

In [3]:
# 데이터 불러오기
train = pd.read_csv('/MainData/train.csv')
test=pd.read_csv('/MainData/test.csv')

In [4]:
train.cat2.value_counts()

음식점       4912
육상 레포츠    2288
자연관광지     1578
역사관광지     1571
숙박시설      1434
문화시설      1220
휴양관광지      889
체험관광지      805
쇼핑         696
축제         569
건축/조형물     328
수상 레포츠     277
관광자원       141
공연/행사      139
산업관광지       93
복합 레포츠      24
항공 레포츠      11
레포츠소개       11
Name: cat2, dtype: int64

In [5]:
train.cat2.unique()

array(['자연관광지', '육상 레포츠', '음식점', '축제', '역사관광지', '문화시설', '휴양관광지', '숙박시설',
       '공연/행사', '쇼핑', '체험관광지', '복합 레포츠', '건축/조형물', '관광자원', '수상 레포츠',
       '산업관광지', '항공 레포츠', '레포츠소개'], dtype=object)

In [6]:
# ! pip install icrawler
from icrawler.builtin import GoogleImageCrawler

In [7]:
# 이미지 크롤링
google_crawler = GoogleImageCrawler(parser_threads=2, downloader_threads=4,
                                    storage={'root_dir': '/tour_data//image/crawling_img/'})

google_crawler.crawl(keyword='여행 명소', max_num=500,
                     min_size=(200,200), max_size=None)
# keyward: 수집하고자 하는 이미지
# max_num: 수집할 이미지 수
# date_min/date_max: 수집할 기간
# min_size/max_size: 이미지 크기

2022-10-05 19:32:54,552 - INFO - icrawler.crawler - start crawling...
2022-10-05 19:32:54,552 - INFO - icrawler.crawler - starting 1 feeder threads...
2022-10-05 19:32:54,552 - INFO - feeder - thread feeder-001 exit
2022-10-05 19:32:54,552 - INFO - icrawler.crawler - starting 2 parser threads...
2022-10-05 19:32:54,567 - INFO - icrawler.crawler - starting 4 downloader threads...
2022-10-05 19:32:55,872 - INFO - parser - parsing result page https://www.google.com/search?q=%EC%97%AC%ED%96%89+%EB%AA%85%EC%86%8C&ijn=0&start=0&tbs=&tbm=isch
2022-10-05 19:32:55,966 - INFO - parser - parsing result page https://www.google.com/search?q=%EC%97%AC%ED%96%89+%EB%AA%85%EC%86%8C&ijn=1&start=100&tbs=&tbm=isch
2022-10-05 19:32:56,154 - INFO - downloader - image #1	https://static.hubzum.zumst.com/hubzum/2018/02/08/13/67608d0ce7cf46959cf9235a93a22375.jpg
2022-10-05 19:32:56,215 - INFO - downloader - image #2	https://static.hubzum.zumst.com/hubzum/2018/02/08/13/c55478f174574c47bcfe70cd6a423379.jpg
2022-1

In [34]:
# 분류 대상 카테고리 선택하기
trip_dir = "/MainData/image/crawling_img"
categories=['자연관광지', '육상 레포츠', '음식점', '축제', '역사관광지', '문화시설', '휴양관광지', '숙박시설',
       '공연/행사', '쇼핑', '체험관광지', '복합 레포츠', '건축/조형물', '관광자원', '수상 레포츠',
       '산업관광지', '항공 레포츠', '레포츠소개']
nb_classes=len(categories)

In [43]:
# 이미지 크기 지정
image_w = 300
image_h = 200
pixels = image_w * image_h * 3

In [47]:
# 이미지 데이터 읽어 들이기 
X = []
Y = []
for idx, cat in enumerate(categories):
    # 레이블 지정 
    label = [0 for i in range(nb_classes)]
    label[idx] = 1
    # 이미지 
    image_dir = trip_dir + '/' + cat
    files = glob.glob(image_dir+"/.jpg")
    for i, f in enumerate(files):
        img = Image.open(f) 
        img = img.convert("RGB")
        img = img.resize((image_w, image_h))
        data = np.asarray(img)      # numpy 배열로 변환
        X.append(data)
        Y.append(label)
        if i % 10 == 0:
            print(i, "\n", data)

In [48]:
X=np.array(X)
Y=np.array(Y)

In [49]:
X, Y

(array([], dtype=float64), array([], dtype=float64))

In [12]:
# 학습, 테스트 데이터 구분
X_train, X_test, y_train, y_test=train_test_split(X, Y,
                                                 stratify=Y,
                                                 random_state=42,
                                                 test_size=0.2)

ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [ ]:
xy=(X_train, X_test, y_train, y_test)

In [ ]:
print('>>> 데이터 저장 중...')
np.save('/MainData/image/crawling_img/7obj.npy', xy)
print('ok', len(Y))